# Exercises: Hands on DNN and 1DCNN

In [1]:
# OS is the built-in python library for exploring the file system
import os 
# Numpy, Pandas and Math are must have libraries
import numpy as np
import pandas as pd
import math

# PIL is the built-in library for trating images; it may be give useful methods as Image, ImageOps 
from PIL import Image, ImageOps 

import tensorflow as tf
import keras

import matplotlib.pyplot as plt
%matplotlib inline

from mpl_toolkits.axes_grid1 import ImageGrid

# GPU usage; see https://www.tensorflow.org/guide/gpu
print('Check on GPU using TensorFlow\n')

gpus = tf.config.list_physical_devices('GPU')

memory_limit = 1024 # 1 Gb
if gpus:
    # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)]  # here the limit
        )
        
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
else:
    print('Found GPU at: {}'.format(device_name))

Check on GPU using TensorFlow

1 Physical GPUs, 1 Logical GPUs
Found GPU at: /device:GPU:0


In [2]:
########################################
# Progress bar
import sys

def printProgressBar(i,max,postText):
    n_bar =10 #size of progress bar
    j= i/max
    sys.stdout.write('\r')
    sys.stdout.write(f"[{'=' * int(n_bar * j):{n_bar}s}] {int(100 * j)}%  {postText}")
    sys.stdout.flush()

# We define two functions that check the type of (sub)strings 
# by raising error if such string cannot be parsed 
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

    
def RepresentsFloat(s):
    try: 
        float(s)
        return True
    except ValueError:
        return False

# After that, we define the function that parses a string into a numpy array
# We will use the Python built in library for Regular Expression, re (see https://docs.python.org/3/library/re.html)
def reinterpret_hist_as_vector(histogram):
    import re

    step = re.split(
        '\[|\]',
        histogram
    )
    step = re.split(
        '\n',
        step[1]
    )
    step = ' '.join(step)
    step = step.split(' ')
    
    hist_els = []
    
    for el in step:
        if RepresentsFloat(el):
            hist_els.append(
                (float(el))
            )
      
    return hist_els

# Here we define a custom function to split our dataset into test/train
def prepare_training_dataset(df_entry, percentage = 0.8):
    # Shuffle the dataframe
    df = df_entry.sample(frac=1).reset_index(drop=True)
    
    # Select percentage
    train_size = math.floor(percentage * len( df ) )
    
    # splitting dataframe by row index 
    df_train = df.iloc[:train_size,:] 
    df_test = df.iloc[train_size:,:] 
    
    # Initializing X,y <- faster version ->
    #X_train = df_train['X'].to_numpy()
    #y_train = df_train['y'].to_numpy()
    #X_test = df_test['X'].to_numpy()
    #y_test = df_test['y'].to_numpy()

    # Initializing X,y 
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    
    
    # Interpret the TRAINING vectors
    print('Preparing the TRAINING dataset...\n')
    for index, row in df_train.iterrows():
        # Progress bar
        percentage = math.floor(index/len(df_train.index) * 100)
        if ( percentage % 10 == 0 ):
            print_train_in_progress_bar = str(percentage) + '%'
            printProgressBar(index, len(df_train.index), print_train_in_progress_bar)
        # Histogram
        X_train.append( row['X'] )
        #RGB 
        y_train.append( row['y'] )
        
    print('\n\nPreparing the TEST dataset...\n')
    # Interpret the TEST vectors    
    for index, row in df_test.iterrows():
        # Progress bar
        percentage = math.floor(((index-len(df_train.index))/len(df_test.index)) * 100)
        if ( percentage % 10 == 0 ):
            print_test_in_progress_bar = str(percentage) + '%'
            printProgressBar(
                index-len(df_train.index), 
                len(df_test.index), 
                print_test_in_progress_bar
            )
        # Histogram 
        X_test.append( row['X'] )
        
        #RGB 
        y_test.append( row['y']  )
    
    print('\nReturining Train/Test datasets\n')
    # returning
    return np.array(X_train), np.array(y_train), np.array(X_test), np.array(y_test)

## Exercise 1: Counteract Overfitting - DropOut layers

<img src="https://gitlab.com/alessandro.bombini.fi/cnn1d_tutorial/-/raw/master/Assets/Images/overfitting_1.png" alt="Drawing" style="float: center; margin-top: 15px"/>

In statistics, overfitting is:

*"the production of an analysis that corresponds too closely or exactly to a particular set of data, and may therefore fail to fit additional data or predict future observations reliably."*

<img src="https://gitlab.com/alessandro.bombini.fi/cnn1d_tutorial/-/raw/master/Assets/Images/overfitting-comics.jpg" alt="Drawing" style="float: center; margin-top: 15px"/>

In machine learning models (which are statistical models), overfitting appears when there is an umbalance between the number of trainable parameters and the size of the training dataset; it becomes manifest during the training process when the model performs poorly on the test/validation dataset, w.r.t. the training dataset, i.e. when the validation scores starts increasing in epochs, while training scores still decreases

<img src="https://gitlab.com/alessandro.bombini.fi/cnn1d_tutorial/-/raw/master/Assets/Images/overfitting_2.png" alt="Drawing" style="float: center; margin-top: 15px; width: 400px"/>

A way to counteract this event in Deep Neural Networks is to add **drop layers** in the DNN architecture. [2], [3], [4]

Dropout is a regularization method that approximates training a large number of neural networks with different architectures in parallel. During training, some number of layer outputs are randomly ignored or “dropped out”. Dropout has the effect of making the training process noisy, forcing nodes within a layer to probabilistically take on more or less responsibility for the inputs.

<img src="https://gitlab.com/alessandro.bombini.fi/cnn1d_tutorial/-/raw/master/Assets/Images/DropLayer.gif" alt="Drawing" style="float: center; margin-top: 15px;"/>

In Keras, dropout layers are asily implemented by the *Dropout* class [5]

tf.keras.layers.Dropout(rate, noise_shape=None, seed=None, **kwargs)

that:

*The Dropout layer randomly sets input units to 0 with a frequency of rate at each step during training time, which helps prevent overfitting. Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over all inputs is unchanged.*

**EXERCISE**

Build a Deep Dense Neural Network with (at least) 5 layers, and apply it to our model. (Keep an eye on the number of parameters, and on how long does it take to train an epoch).

See if it shows overfitting; if not, deepen it, until it shows it. 

Then, create a new network with Dense layers, and see if it is now free of overfitting. 

-------
[1] https://www.lexico.com/definition/overfitting

[2] https://machinelearningmastery.com/dropout-for-regularizing-deep-neural-networks/

[3] https://machinelearningmastery.com/how-to-reduce-overfitting-with-dropout-regularization-in-keras/

[4] N. Srivastava, G. Hinton, A. Krizhevsky, I. Sutskever, R. Salakhutdinov, *Dropout: A Simple Way to Prevent Neural Networks from Overfitting* https://jmlr.org/papers/v15/srivastava14a.html

[5] https://keras.io/api/layers/regularization_layers/dropout/

## Exercise 2: Deep CNN; hints from ImageNet Large Scale Visual Recognition Challenge winners - AlexNet & VGG

The **ImageNet** project is a large visual database designed for use in visual object recognition software research. More than 14 million images have been hand-annotated by the project. 

Since 2010, a competition was launch, dubbed *ImageNet Large Scale Visual Recognition Challenge* (ILSVRC), where researchers from all around the globe have to design a novel Deep Learning architecture to label the dataset. From the ImageNet homepage [1]:

*The ImageNet Large Scale Visual Recognition Challenge (ILSVRC) evaluates algorithms for object detection and image classification at large scale. One high level motivation is to allow researchers to compare progress in detection across a wider variety of objects -- taking advantage of the quite expensive labeling effort. Another motivation is to measure the progress of computer vision for large scale image indexing for retrieval and annotation.*

### AlexNet

On 30 September 2012, a convolutional neural network (CNN) called **AlexNet** [3] achieved a top-5 error of 15.3% in the ImageNet 2012 Challenge, more than 10.8 percentage points lower than that of the runner up. 

AlexNet is considered one of the most influential papers published in computer vision, having spurred many more papers published employing CNNs and GPUs to accelerate deep learning. As of 2021, the AlexNet paper has been cited over 80,000 times according to Google Scholar. 

<img src="https://gitlab.com/alessandro.bombini.fi/cnn1d_tutorial/-/raw/master/Assets/Images/AlexNet.png" alt="Drawing" style="float: center; margin-top: 15px"/>

AlexNet contained eight layers; the first five were convolutional layers, some of them followed by max-pooling layers, and the last three were fully connected layers. It used the non-saturating ReLU activation function, which showed improved training performance over tanh and sigmoid. In detail [3]:

- 1 Convolutional layer with kernel size (11,11) and 96 filters, followed by a (3,3)-MaxPooling;
- 1 Convolutional layer with kernel size (5,5) and 256 filters, followed by a (3,3)-MaxPooling;
- 1 Convolutional layer with kernel size (3,3) and 128 filters;
- 2 Convolutional layers with kernel size (3,3) and 192 filters each, followed by a Flatten layer to feed the dense layer;
- 2 Dense layers with 4096 neurons and a 0.5-probability drop layer (see ex.1)
- the Output layer which may label 10 categories.

In code [4]: 

```
AlexNet_model = keras.models.Sequential(
    [
    # 1°: (11x11)
    keras.layers.Conv2D(filters=96, 
                            kernel_size=(11,11), strides=(4,4), activation='relu', 
                            input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),

    # 2° (5x5)
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),

    # 3,4,5° (3x3)
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization()
    ,
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),

    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),

    # Dense layers
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),

    # Output
    keras.layers.Dense(10, activation='softmax')
    ]
)
```

### VGG

In 2014, a novel architecture developed by the Oxford's *Visual Geometry Group* (VGG), dubbed *Very Deep Convolutional Networks for Large-Scale Visual Recognition* (shortened in VGG-16) was presented at ILSVRC; citing the VGG page [5] 

*Our main contribution is a rigorous evaluation of networks of increasing depth, which shows that a significant improvement on the prior-art configurations can be achieved by increasing the depth to 16-19 weight layers, which is substantially deeper than what has been used in the prior art. To reduce the number of parameters in such very deep networks, we use very small 3×3 filters in all convolutional layers (the convolution stride is set to 1). Please see our publication for more details.*

*The very deep ConvNets were the basis of our ImageNet ILSVRC-2014 submission, where our team (VGG) secured the first and the second places in the localisation and classification tasks respectively.*

<img src="https://gitlab.com/alessandro.bombini.fi/cnn1d_tutorial/-/raw/master/Assets/Images/VGG163d.png" alt="Drawing" style="float: center; margin-top: 15px"/>

The VGG architecture (in his base form, VGG-16, and its evolution, VGG-19) is [6]:

<img src="https://gitlab.com/alessandro.bombini.fi/cnn1d_tutorial/-/raw/master/Assets/Images/VGG1619.png" alt="Drawing" style="float: center; margin-top: 15px; width: 50%"/>

### EXERCISE:

Choose one of the two architectures, and write down an analougus 1-dimensional model. Keep the numbers low, you should avoid either overfitting (see ex.1) as well as a too long training time. 

After that, try adapting the various (hyper)parameters to lower the loss, and, when you find something plausible, try to recolor the XRF image, as in the lecture. 

-------------------
[1] https://image-net.org/challenges/LSVRC/

[2] O. Russakovsky et al., *ImageNet Large Scale Visual Recognition Challenge* https://arxiv.org/abs/1409.0575

[3] A. Krizhevsky, I. Sutskever, G. E. Hinton, *ImageNet Classification with Deep ConvolutionalNeural Networks*, doi:10.1145/3065386 https://papers.nips.cc/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf

[4] https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98

[5] https://www.robots.ox.ac.uk/~vgg/research/very_deep/

[6] K. Simonyan, A. Zisserman, *Very Deep Convolutional Networks for Large-Scale Image Recognition*, https://arxiv.org/abs/1409.1556

For an extensive recap on ILSVRC and more: https://medium.com/analytics-vidhya/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5

### AlexNet

### VGG-16

## Exercise 3: Design your own DNN

**EXERCISE**

Build your own Deep Neural Network. Use anything you like from the lecture. 

PS: a suggestion: try to use the keras functional API, and a non-linear topology. Google "Deep Learning" + a title of a famous Christopher Nolan's movie with Di Caprio and Joseph Gordon-Levitt. It could give you inspiration. 